In [1]:
#Import lib
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta


In [2]:
#Load Data
tickers = ['GEV', 'TGTX', 'PRVA', 'CEVA', 'GILD', 'KD', 'NRG']
bench_data = ['^GSPC', '^DJI', 'IWM','SPY']

end_date = datetime.now()
start_date = end_date - \
            timedelta(days=1095) # 3 years of data

tickers = yf.download(tickers, start = start_date, end = end_date,auto_adjust=False)
bench_data = yf.download (bench_data, start = start_date, end = end_date,auto_adjust=False)     

print(tickers)
print(bench_data)


[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  4 of 4 completed


Price       Adj Close                                                 \
Ticker           CEVA         GEV        GILD         KD         NRG   
Date                                                                   
2022-03-30  40.380001         NaN   53.077934  13.150000   34.552280   
2022-03-31  40.650002         NaN   52.891102  13.120000   34.861275   
2022-04-01  41.529999         NaN   53.086830  13.260000   34.897629   
2022-04-04  40.840000         NaN   53.976498  13.530000   34.234203   
2022-04-05  38.980000         NaN   53.158009  12.970000   34.188766   
...               ...         ...         ...        ...         ...   
2025-03-24  28.870001  343.570007  106.739998  35.320000  102.860001   
2025-03-25  28.110001  336.200012  107.889999  34.970001  102.269997   
2025-03-26  27.410000  317.700012  109.269997  34.060001   97.370003   
2025-03-27  27.010000  303.000000  111.160004  32.320000   95.760002   
2025-03-28  25.940001  302.929993  111.790001  31.320000   95.19

In [3]:
portfolio_weights = np.ones(len(tickers)) / len(tickers)
portfolio_returns = tickers.pct_change()
portfolio_volatility = portfolio_returns.std() * np.sqrt(252) #annualized volatility

In [4]:
def calc_beta (bench_data, window = 12):
    np_array = bench_data.values
    m = np_array [:,0]
    beta = []
    for ind, col in enumerate(bench_data):
        if ind > 0:
            s = np_array[:, ind]
            covariance = np.cov(s,m)
            beta.append( covariance [0,1] / covariance [1,1])
    return pd.Series(beta, bench_data.columns[1:], name = 'Beta')

bench_data = pd.DataFrame(bench_data)
beta_values = calc_beta(bench_data, window = 12)
print(beta_values)


Price      Ticker
Adj Close  SPY       3.687820e+00
           ^DJI      1.986107e+02
           ^GSPC     3.527833e+01
Close      IWM       9.216159e-01
           SPY       3.516441e+00
           ^DJI      1.986107e+02
           ^GSPC     3.527833e+01
High       IWM       9.205416e-01
           SPY       3.505948e+00
           ^DJI      1.980884e+02
           ^GSPC     3.518379e+01
Low        IWM       9.228899e-01
           SPY       3.519799e+00
           ^DJI      1.987508e+02
           ^GSPC     3.531284e+01
Open       IWM       9.212140e-01
           SPY       3.515413e+00
           ^DJI      1.980518e+02
           ^GSPC     3.523999e+01
Volume     IWM      -6.034474e+04
           SPY      -7.932891e+05
           ^DJI      2.357259e+06
           ^GSPC    -3.736731e+06
Name: Beta, dtype: float64


In [5]:
portfolio_drawdowns = (tickers / tickers.rolling(window = 252).max()) - 1
pportfolio_avg_drawdown = portfolio_drawdowns.mean()
portfolio_max_drawdown = portfolio_drawdowns.min()
print(portfolio_max_drawdown)
print(pportfolio_avg_drawdown)



Price      Ticker
Adj Close  CEVA     -0.528862
           GEV            NaN
           GILD     -0.265898
           KD       -0.325527
           NRG      -0.310608
           PRVA     -0.448154
           TGTX     -0.809143
Close      CEVA     -0.528862
           GEV            NaN
           GILD     -0.276549
           KD       -0.325527
           NRG      -0.337460
           PRVA     -0.448154
           TGTX     -0.809143
High       CEVA     -0.527418
           GEV            NaN
           GILD     -0.272676
           KD       -0.312028
           NRG      -0.338770
           PRVA     -0.449597
           TGTX     -0.808803
Low        CEVA     -0.529581
           GEV            NaN
           GILD     -0.279010
           KD       -0.317761
           NRG      -0.346187
           PRVA     -0.448970
           TGTX     -0.808252
Open       CEVA     -0.537952
           GEV            NaN
           GILD     -0.273906
           KD       -0.311642
           NRG      -0

In [6]:
portfolio_total_returns = (tickers.iloc[-1] / tickers.iloc[0]) - 1
portfolio_annualized_returns = ((1 + portfolio_total_returns) ** (1 /10)) - 1 # Assuming 10 years

In [7]:
risk_analysis = pd.DataFrame({
    'Ticker': tickers,
    'Portfolio Weight': portfolio_weights,
    'Annualized Returns': portfolio_volatility,
    'Beta against ^GSPC': beta_values['^GSPC'],
    'Beta against ^DJI': beta_values['^DJI'],
    'Beta against IWM': beta_values['IWM'],
    'Average Drawdown': pportfolio_avg_drawdown,
    'Max Drawdown': portfolio_max_drawdown,
    'Total Returns': portfolio_total_returns,
    'Annualized Returns': portfolio_annualized_returns
})

KeyError: '^GSPC'

In [ ]:
Portfolio_Correlation = portfolio_returns.corr(bench_data)
Portfolio_Covariance = portfolio_returns.cov(bench_data)
Portfolio_Tracking_Error = np.sqrt(portfolio_covariance.diagonal())
Portfolio_Sharpe_Ratio = (portfolio_total_returns - 0.02) / portfolio_volatility #Assuming Risk-Free Rate of 2

In [ ]:
bench_volatility = bench_data.pct_change().std() * np.sqrt(252)
volatility_spread = portfolio_volatility - bench_volatility

In [ ]:
Portfolio_risk_table = pd.DataFrame({
    'ETF Ticker': bench_data,
    'Correlation against ETF': Portfolio_Correlation,
    'Covariance against ETF': Portfolio_Covariance,
    'Tracking Error': Portfolio_Tracking_Error,
    'Sharpe Ratio': Portfolio_Sharpe_Ratio,
    'Annualized Volatility Spread': bench_volatility,
})

In [ ]:
correlation_matrix = pd.concat([portfolio_returns, bench_data], axis = 1).corr()


In [ ]:
print("Portfolio Risk Analysis")
print(risk_analysis)

print("\nPortfolio Risk against Benchmarks")
print(Portfolio_risk_table)

print("\nCorrelation Matrix")
print(correlation_matrix)

In [ ]:
#Daily Returns
def calculate_daily_returns(tickers):
    daily_returns = tickers.pct_change()
    return daily_returns

#Cumulative Returns
def cumulative_returns_returns(data):
    return (1+data.pct.change()).cumprod() -1
    

#Max Sharpe Ratio Portfolio
def maximum_sharpe_drawdown(cumulative_returns):
    cov_matrix = returns.cov()
    inv_cov_matrix = np.linalg.inv(cov_matrix)
    ones_vector = np.ones(len(returns.columns))
    weights = inv_cov_matrix.dot(ones_vector) / ones_vector.dot(inv_cov_matrix).do
    return returns.dot(weights)
    
#Risk Parity Portfolio
def risk_parity_portfolio(returns):
    inv_cov_matrix = np.linalg.inv(returns.cov())
    weights = inv_cov_matrix.dot(np.ones(len(returns.columns))) / len(returns.columns)
    return returns.dot(weights)

#Inverse Volatility Weighted Portfolio
def inverse_volatility_weighted_portfolio(returns):
    weights = 1 / returns.std()
    weights /= weights.sum()
    return returns.dot(weights)
